In [36]:
import glob
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics
import pandas as pd
import pickle

In [2]:
PROXY_REWARD_TAGS = "custom_metrics/proxy_reward_mean"
TRUE_REWARD_TAGS = "custom_metrics/true_reward_mean"
OM_TAGS = "info/learner/safe_policy0/occupancy_measure_kl"
AD_TAGS = "info/learner/safe_policy0/action_distribution_kl"

In [3]:
GLUCOSE_SAFE_POLICY_PROXY = -106.646103
GLUCOSE_SAFE_POLICY_TRUE = -72635.791604
GLUCOSE_NO_REG_PROXY = -0.91035673333333
GLUCOSE_NO_REG_TRUE = -599023.451246629
GLUCOSE_THRESHOLD_PROXY = GLUCOSE_SAFE_POLICY_PROXY  # np.mean([GLUCOSE_NO_REG_PROXY, GLUCOSE_SAFE_POLICY_PROXY])
GLUCOSE_THRESHOLD_TRUE = (
    GLUCOSE_SAFE_POLICY_TRUE  # np.mean([GLUCOSE_NO_REG_TRUE, GLUCOSE_SAFE_POLICY_TRUE])
)

PANDEMIC_SAFE_POLICY_PROXY = -12.057395
PANDEMIC_SAFE_POLICY_TRUE = -12.257395
PANDEMIC_NO_REG_PROXY = -1.2893852358693285
PANDEMIC_NO_REG_TRUE = -29.57349266440372
PANDEMIC_THRESHOLD_PROXY = PANDEMIC_SAFE_POLICY_PROXY  # np.mean([PANDEMIC_NO_REG_PROXY, PANDEMIC_SAFE_POLICY_PROXY])
PANDEMIC_THRESHOLD_TRUE = PANDEMIC_SAFE_POLICY_TRUE  # np.mean([PANDEMIC_NO_REG_TRUE, PANDEMIC_SAFE_POLICY_TRUE])

TOMATO_SAFE_POLICY_PROXY = 6.142000
TOMATO_SAFE_POLICY_TRUE = 5.856167
TOMATO_NO_REG_PROXY = 45.724166666666726
TOMATO_NO_REG_TRUE = 2.3516666666666683
TOMATO_THRESHOLD_PROXY = (
    TOMATO_SAFE_POLICY_PROXY  # np.mean([TOMATO_NO_REG_PROXY, TOMATO_SAFE_POLICY_PROXY])
)
TOMATO_THRESHOLD_TRUE = (
    TOMATO_SAFE_POLICY_TRUE  # np.mean([TOMATO_NO_REG_TRUE, TOMATO_SAFE_POLICY_TRUE])
)

TRAFFIC_SAFE_POLICY_PROXY = 1360.404346
TRAFFIC_SAFE_POLICY_TRUE = -2284.477325
TRAFFIC_NO_REG_PROXY = 2569.5555456780753
TRAFFIC_NO_REG_TRUE = -57377.787423069814
TRAFFIC_THRESHOLD_PROXY = TRAFFIC_SAFE_POLICY_PROXY  # np.mean([TRAFFIC_NO_REG_PROXY, TRAFFIC_SAFE_POLICY_PROXY])
TRAFFIC_THRESHOLD_TRUE = (
    TRAFFIC_SAFE_POLICY_TRUE  # np.mean([TRAFFIC_NO_REG_TRUE, TRAFFIC_SAFE_POLICY_TRUE])
)

# Setup of progress files

In [4]:
# Glucose
files_to_evaluate_glucose_om = glob.glob(
    "/nas/ucb/shivamsinghal/occupancy-measure-anomaly-detection/data/logs/glucose/ORPO/*/model_256-256/ICML-bcinit/OM/*/*/*/*/checkpoint_000500"
)
files_to_evaluate_glucose_policy = glob.glob(
    "/nas/ucb/shivamsinghal/occupancy-measure-anomaly-detection/data/logs/glucose/ORPO/*/model_256-256/ICML-bcinit/AD/*/*/*/checkpoint_000500"
)
files_to_evaluate_glucose = (
    files_to_evaluate_glucose_om + files_to_evaluate_glucose_policy
)
glucose_events = [
    f'{g[:g.rindex("/")]}/progress.csv' for g in files_to_evaluate_glucose
]
glucose_events = [pd.read_csv(glob.glob(g)[0]) for g in glucose_events]

# Pandemic
files_to_evaluate_pandemic_om = glob.glob(
    "/nas/ucb/shivamsinghal/occupancy-measure-anomaly-detection/data/logs/pandemic/ORPO/*/model_128-128/ICML-bcinit/OM/*/weights_10.0_0.1_0.01/*/*/*/checkpoint_000260"
)
file_to_evaluate_pandemic_policy = glob.glob(
    "/nas/ucb/shivamsinghal/occupancy-measure-anomaly-detection/data/logs/pandemic/ORPO/proxy/model_128-128/ICML-bcinit/AD/weights_10.0_0.1_0.01/*/*/*/checkpoint_000260"
)
files_to_evaluate_pandemic = (
    file_to_evaluate_pandemic_policy + files_to_evaluate_pandemic_om
)
pandemic_events = [
    f'{g[:g.rindex("/")]}/progress.csv' for g in files_to_evaluate_pandemic
]
pandemic_events = [pd.read_csv(glob.glob(g)[0]) for g in pandemic_events]

# Tomato
files_to_evaluate_tomato_policy = glob.glob(
    "/nas/ucb/shivamsinghal/occupancy-measure-anomaly-detection/data/logs/tomato/rhard/ORPO/*/model_512-512-512-512/ICML-rand/AD/new_policy/*/*/*/checkpoint_000500"
)
files_to_evaluate_tomato_om = glob.glob(
    "/nas/ucb/shivamsinghal/occupancy-measure-anomaly-detection/data/logs/tomato/rhard/ORPO/*/model_512-512-512-512/ICML-rand/OM/*/new_policy/*/*/*/checkpoint_000500"
)
files_to_evaluate_tomato = files_to_evaluate_tomato_policy + files_to_evaluate_tomato_om
tomato_events = [f'{g[:g.rindex("/")]}/progress.csv' for g in files_to_evaluate_tomato]
tomato_events = [pd.read_csv(glob.glob(g)[0]) for g in tomato_events]

# Traffic
files_to_evaluate_traffic = glob.glob(
    "/nas/ucb/cassidy/occupancy-measures/data/logs/traffic/singleagent_merge_bus/ORPO/proxy/model_512-512-512-512/icml/*/*/seed_[0-4]/*/progress.csv"
)
traffic_events = [pd.read_csv(f) for f in files_to_evaluate_traffic]

In [5]:
def split_and_average(array, num_splits=10):
    split_arrays = np.array_split(array, num_splits)
    averages = [np.mean(split) for split in split_arrays]
    return averages

In [6]:
def index_of_max_average_change(data):
    moving_averages = [(data[i] + data[i + 1]) / 2 for i in range(len(data) - 1)]
    changes = [
        abs(moving_averages[i + 1] - moving_averages[i])
        for i in range(len(moving_averages) - 1)
    ]
    max_change_index = changes.index(max(changes))
    return moving_averages[max_change_index]

In [7]:
def max_accuracy(scores, labels):
    possible_accuracies = np.equal(
        scores[None, :] > scores[:, None],
        labels[None, :],
    ).mean(axis=1)
    max_accuracy = possible_accuracies.max()
    return max_accuracy

In [8]:
def get_stacked_results(files):
    proxy = [split_and_average(list(d[PROXY_REWARD_TAGS])) for d in files]
    true = [split_and_average(list(d[TRUE_REWARD_TAGS])) for d in files]
    om = [split_and_average(list(d[OM_TAGS])) for d in files]
    ad = [split_and_average(list(d[AD_TAGS])) for d in files]
    return np.stack(proxy), np.stack(true), np.stack(om), np.stack(ad)

In [19]:
glucose_proxy_stack, glucose_true_stack, glucose_om_stack, glucose_ad_stack = (
    get_stacked_results(glucose_events)
)
pandemic_proxy_stack, pandemic_true_stack, pandemic_om_stack, pandemic_ad_stack = (
    get_stacked_results(pandemic_events)
)
traffic_proxy_stack, traffic_true_stack, traffic_om_stack, traffic_ad_stack = (
    get_stacked_results(traffic_events)
)
tomato_proxy_stack, tomato_true_stack, tomato_om_stack, tomato_ad_stack = (
    get_stacked_results(tomato_events)
)

/nas/ucb/k8/shivamsinghal/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/nas/ucb/k8/shivamsinghal/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [21]:
# traffic files have some null data
traffic_om_stack = np.delete(
    traffic_om_stack, np.where(np.isnan(traffic_ad_stack))[0], axis=0
)
traffic_proxy_stack = np.delete(
    traffic_proxy_stack, np.where(np.isnan(traffic_ad_stack))[0], axis=0
)
traffic_true_stack = np.delete(
    traffic_true_stack, np.where(np.isnan(traffic_ad_stack))[0], axis=0
)
traffic_ad_stack = np.delete(
    traffic_ad_stack, np.where(np.isnan(traffic_ad_stack))[0], axis=0
)

traffic_ad_stack = np.delete(
    traffic_ad_stack, np.where(np.isnan(traffic_om_stack))[0], axis=0
)
traffic_proxy_stack = np.delete(
    traffic_proxy_stack, np.where(np.isnan(traffic_om_stack))[0], axis=0
)
traffic_true_stack = np.delete(
    traffic_true_stack, np.where(np.isnan(traffic_om_stack))[0], axis=0
)
traffic_om_stack = np.delete(
    traffic_om_stack, np.where(np.isnan(traffic_om_stack))[0], axis=0
)

traffic_ad_stack = np.delete(
    traffic_ad_stack, np.where(np.isnan(traffic_proxy_stack))[0], axis=0
)
traffic_om_stack = np.delete(
    traffic_om_stack, np.where(np.isnan(traffic_proxy_stack))[0], axis=0
)
traffic_true_stack = np.delete(
    traffic_true_stack, np.where(np.isnan(traffic_proxy_stack))[0], axis=0
)
traffic_proxy_stack = np.delete(
    traffic_proxy_stack, np.where(np.isnan(traffic_proxy_stack))[0], axis=0
)

traffic_ad_stack = np.delete(
    traffic_ad_stack, np.where(np.isnan(traffic_true_stack))[0], axis=0
)
traffic_om_stack = np.delete(
    traffic_om_stack, np.where(np.isnan(traffic_true_stack))[0], axis=0
)
traffic_proxy_stack = np.delete(
    traffic_proxy_stack, np.where(np.isnan(traffic_true_stack))[0], axis=0
)
traffic_true_stack = np.delete(
    traffic_true_stack, np.where(np.isnan(traffic_true_stack))[0], axis=0
)

In [26]:
# def get_thresholds(safe_policy_true, safe_policy_proxy, events, evaluation_files, no_reg_tag="state-action/kl-0/"):
#     no_regularization_runs = [events[i] for i in range(len(evaluation_files)) if no_reg_tag in evaluation_files[i] and "proxy" in evaluation_files[i]]
#     true_reward_no_reg = np.median([run[TRUE_REWARD_TAGS].iat[-1] for run in no_regularization_runs])
#     proxy_reward_no_reg = np.median([run[PROXY_REWARD_TAGS].iat[-1] for run in no_regularization_runs])
#     print(true_reward_no_reg)
#     return np.mean([true_reward_no_reg, safe_policy_true]), np.mean([proxy_reward_no_reg, safe_policy_proxy])

# Analyze data and Calculate metrics

In [34]:
def get_auroc_and_max_accuracy(
    proxy_stack, proxy_threshold, true_stack, true_threshold, ad, om
):
    reward_hacking = np.logical_and(
        proxy_stack > proxy_threshold, true_stack < true_threshold
    ).astype(int)
    reward_hacking_concat = np.concatenate(reward_hacking).ravel()
    om_concat = np.concatenate(om).ravel()
    ad_concat = np.concatenate(ad).ravel()

    fpr_om, tpr_om, thresholds = metrics.roc_curve(reward_hacking_concat, om_concat)
    roc_auc_om = metrics.auc(fpr_om, tpr_om)
    fpr_ad, tpr_ad, thresholds = metrics.roc_curve(reward_hacking_concat, ad_concat)
    roc_auc_ad = metrics.auc(fpr_ad, tpr_ad)

    om_max_accuracy = max_accuracy(om_concat, reward_hacking_concat)
    ad_max_accuracy = max_accuracy(ad_concat, reward_hacking_concat)

    return (
        {"fpr": fpr_om, "tpr": tpr_om, "roc_auc": roc_auc_om},
        {"fpr": fpr_ad, "tpr": tpr_ad, "roc_auc": roc_auc_ad},
        om_max_accuracy,
        ad_max_accuracy,
    )

Glucose

In [37]:
(
    glucose_roc_auc_om,
    glucose_roc_auc_ad,
    glucose_om_max_accuracy,
    glucose_ad_max_accuracy,
) = get_auroc_and_max_accuracy(
    glucose_proxy_stack,
    GLUCOSE_THRESHOLD_PROXY,
    glucose_true_stack,
    GLUCOSE_THRESHOLD_TRUE,
    glucose_ad_stack,
    glucose_om_stack,
)

with open("glucose_roc_auc_om.pickle", "wb") as handle:
    pickle.dump(glucose_roc_auc_om, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open("glucose_roc_auc_ad.pickle", "wb") as handle:
    pickle.dump(glucose_roc_auc_ad, handle, protocol=pickle.HIGHEST_PROTOCOL)

Pandemic

In [39]:
(
    pandemic_roc_auc_om,
    pandemic_roc_auc_ad,
    pandemic_om_max_accuracy,
    pandemic_ad_max_accuracy,
) = get_auroc_and_max_accuracy(
    pandemic_proxy_stack,
    PANDEMIC_THRESHOLD_PROXY,
    pandemic_true_stack,
    PANDEMIC_THRESHOLD_TRUE,
    pandemic_ad_stack,
    pandemic_om_stack,
)

with open("pandemic_roc_auc_om.pickle", "wb") as handle:
    pickle.dump(pandemic_roc_auc_om, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open("pandemic_roc_auc_ad.pickle", "wb") as handle:
    pickle.dump(pandemic_roc_auc_ad, handle, protocol=pickle.HIGHEST_PROTOCOL)

Traffic

In [41]:
(
    traffic_roc_auc_om,
    traffic_roc_auc_ad,
    traffic_om_max_accuracy,
    traffic_ad_max_accuracy,
) = get_auroc_and_max_accuracy(
    traffic_proxy_stack,
    TRAFFIC_THRESHOLD_PROXY,
    traffic_true_stack,
    TRAFFIC_THRESHOLD_TRUE,
    traffic_ad_stack,
    traffic_om_stack,
)

with open("traffic_roc_auc_om.pickle", "wb") as handle:
    pickle.dump(traffic_roc_auc_om, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open("traffic_roc_auc_ad.pickle", "wb") as handle:
    pickle.dump(traffic_roc_auc_ad, handle, protocol=pickle.HIGHEST_PROTOCOL)

Tomato

In [42]:
tomato_roc_auc_om, tomato_roc_auc_ad, tomato_om_max_accuracy, tomato_ad_max_accuracy = (
    get_auroc_and_max_accuracy(
        tomato_proxy_stack,
        TOMATO_THRESHOLD_PROXY,
        tomato_true_stack,
        TOMATO_THRESHOLD_TRUE,
        tomato_ad_stack,
        tomato_om_stack,
    )
)

with open("tomato_roc_auc_om.pickle", "wb") as handle:
    pickle.dump(tomato_roc_auc_om, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open("tomato_roc_auc_ad.pickle", "wb") as handle:
    pickle.dump(tomato_roc_auc_ad, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Tabulate Results

In [46]:
data_cols = ["env", "OM AUROC", "AD AUROC", "OM max accuracy", "AD max accuracy"]

In [54]:
tomato_data = [
    "Tomato",
    tomato_roc_auc_om["roc_auc"],
    tomato_roc_auc_ad["roc_auc"],
    tomato_om_max_accuracy,
    tomato_ad_max_accuracy,
]
traffic_data = [
    "Traffic",
    traffic_roc_auc_om["roc_auc"],
    traffic_roc_auc_ad["roc_auc"],
    traffic_om_max_accuracy,
    traffic_ad_max_accuracy,
]
glucose_data = [
    "Glucose",
    glucose_roc_auc_om["roc_auc"],
    glucose_roc_auc_ad["roc_auc"],
    glucose_om_max_accuracy,
    glucose_ad_max_accuracy,
]
pandemic_data = [
    "Pandemic",
    pandemic_roc_auc_om["roc_auc"],
    pandemic_roc_auc_ad["roc_auc"],
    pandemic_om_max_accuracy,
    pandemic_ad_max_accuracy,
]

data = [tomato_data, traffic_data, glucose_data, pandemic_data]

In [55]:
data_df = pd.DataFrame(data, columns=data_cols)

In [56]:
data_df = data_df.set_index("env")

In [57]:
data_df.to_latex()

/tmp/ipykernel_916993/2218615829.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  data_df.to_latex()


'\\begin{tabular}{lrrrr}\n\\toprule\n{} &  OM AUROC &  AD AUROC &  OM max accuracy &  AD max accuracy \\\\\nenv      &           &           &                  &                  \\\\\n\\midrule\nTomato   &  0.995784 &  0.888248 &         0.965022 &         0.856951 \\\\\nTraffic  &  0.995171 &  0.981247 &         0.966163 &         0.919335 \\\\\nGlucose  &  0.991331 &  0.785263 &         0.950000 &         0.742273 \\\\\nPandemic &  0.936118 &  0.821251 &         0.896364 &         0.752727 \\\\\n\\bottomrule\n\\end{tabular}\n'

In [61]:
# Adjusting the LaTeX table string to fit a single column in a two-column paper with smaller font size
latex_table_single_column = """
\\begin{table}[ht]
\\centering
\\caption{Summary of Environment Performance}
\\label{table:environment_performance}
\\small 
\\begin{tabular}{@{}lcccc@{}}
\\hline
Environment & OM AUROC & AD AUROC & OM Max Acc. & AD Max Acc. \\\\
\\hline
"""
for row in data:
    formatted_row = [format_2sf(item) for item in row]
    latex_table_single_column += " & ".join(formatted_row) + " \\\\\n"
latex_table_single_column += """\\hline
\\end{tabular}
\\end{table}
"""

print(latex_table_single_column)


\begin{table}[ht]
\centering
\caption{Summary of Environment Performance}
\label{table:environment_performance}
\small 
\begin{tabular}{@{}lcccc@{}}
\hline
Environment & OM AUROC & AD AUROC & OM Max Acc. & AD Max Acc. \\
\hline
Tomato & 1.00 & 0.89 & 0.97 & 0.86 \\
Traffic & 1.00 & 0.98 & 0.97 & 0.92 \\
Glucose & 0.99 & 0.79 & 0.95 & 0.74 \\
Pandemic & 0.94 & 0.82 & 0.90 & 0.75 \\
\hline
\end{tabular}
\end{table}

